In [1]:
from langchain import hub
import bs4
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaLLM
import ollama  # Assuming Ollama is the interface for using phi3 embeddings
strainer = bs4.SoupStrainer(['article', 'main', 'div', 'section'])
# Load, chunk, and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://www.redblock.ai/blog/a-quick-introduction-to-ai-agents",),
    bs_kwargs=dict(
        parse_only=strainer
    ),
)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

class Phi3Embeddings:
    def embed_documents(self, texts):
        return [ollama.embeddings(model="phi3.5", prompt=text)["embedding"] for text in texts]

    def embed_query(self, text):
        return ollama.embeddings(model="phi3.5", prompt=text)["embedding"]

phi3_embedding_instance = Phi3Embeddings()


vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=phi3_embedding_instance
)



# Create a retriever from the vectorstore

retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={'k': 4, 'fetch_k': 20})
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)





USER_AGENT environment variable not set, consider setting it to identify your requests.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [2]:
llm = OllamaLLM(model="phi3.5")

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [3]:

response = rag_chain.invoke("What does it convey? Summarize it in 3 lines.")

print(response)


Number of requested results 20 is greater than number of elements in index 6, updating n_results = 6


 The document discusses the evolution, capabilities, and impacts of artificial intelligence (AI) agents in various domains such as cybersecurity, daily life convenience through smart devices like Siri or Alexa, direct platform interaction for task automation, memory retention from past interactions to provide personalized assistance.

1. AI agents have evolved beyond simple tasks into complex multi-step processes with adaptive learning capabilities and long-term memory functions that mimic human cognition more closely (dynamic behavior).
2. They are integrated directly in platforms, enhancing efficiency by automating routine activities without the need for constant user intervention (enhancements in task automation & direct interaction).
3. AI agents play a significant role in cybersecurity at RedBlock to identify and prevent advanced threats autonomously, showcasing their importance beyond consumer convenience into critical security applications (application in cybersecurity).


In [4]:
vectorstore.delete_collection()
